In [1]:
import os
os.chdir('C:/Users/solei/Documents/Loss_Run/')
from Inv_Loss_Link import Inventory_Loss

In [3]:
cwd='C:/Users/solei/Documents/Loss_Run/'
# First, instantiate the loss modeul and download sample files if needed
Loss=Inventory_Loss(cwd,Download=False)

# Then, establish the dictionary to map structure configurations to a trivial parameter "Z",
# and then malke a dictionary map Z to global damage ratio given a wind speed and flood depth.
Loss.Define_new_table(cwd_look='ProbDS_cwfd_MaxIter3000.pkl')

# Add a column to the inventory that contains a "Z" value for each row to identify losses in next parts
Loss.Inv_Z_Transformation(cwd_inv='NC_Inventory_v1_AI_Imputed',blgtype_col=['bldgtype',['W']],roof_col=['roof_shape_1',['Cross Gable', 'Complex Gable','Gable'],['Hip','Cross Hip']],story_col=['Story_AI',[1],[2]],garage_col=['Garage',[0],[1]],res_col=['PRIM_OCC',['Single Family Dwelling']],year_col=['med_yr_blt'],Coast_D_col=['Coast_D (miles)',[1]],val_struct_col=['val_struct'],Mode='')

# Match hazard intensities at building level
Loss.Hazard_Inv_map(cwd_inv='NC_Inventory_v1_AI_Imputed',cwd_haz='TC_Wise',W_ind=3,I_ind=4,Tr_F=39.3701,Tr_W=2.23694,Average_Mode='Inverse_d4',Mode='')

# Estimate losses
Loss.Loss_estimate(cwd_haz='TC_Wise',cwd_inv='NC_Inventory_v1_AI_Imputed',zone='leaphi_zones_20230920')

Probabilities summation (should be one; small numerical errors are fine): 1.0000000000000004
Dictionary to map inventory to configurations is created at:	C:/Users/solei/Documents/Loss_Run/Intermediate_outputs\lookup_N.pkl
New look-up table (configuration (Z) -> global loss ratio) is created at:	C:/Users/solei/Documents/Loss_Run/Intermediate_outputs\Z_transform1.pkl
Updated inventory (filtered for archetypes and updated with Z values) stored at:	 C:/Users/solei/Documents/Loss_Run/Intermediate_outputs\NC_Inventory_v1_AI_Imputed_Z.parquet


Hazard instances::   0%|          | 0/3 [00:00<?, ?it/s]

Hazard instances::   0%|          | 0/3 [00:00<?, ?it/s]

Inventory-level are stored at:	 C:/Users/solei/Documents/Loss_Run/Loss_estimates\TC_Wise\Inventory\NC_Inventory_v1_AI_Imputed
zone-level are stored at:	 C:/Users/solei/Documents/Loss_Run/Loss_estimates\TC_Wise\Inventory\NC_Inventory_v1_AI_Imputed\Zones\leaphi_zones_20230920


In [9]:
self=Loss 
self.mk_dir(os.path.join(cwd,'Loss_estimates'))

In [7]:
self=Loss
os.mkdir(os.path.join(self.cwd,'Loss_estimates'))
os.path.join(self.cwd,'Loss_estimates')

FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'C:/Users/solei/Documents/Loss_Run/Loss_estimates'

In [9]:
import geopandas as gpd 
ff=gpd.read_parquet(os.path.join(cwd,'Test','Hazard_match_sample.parquet'))
ff['val_struct_col']=0
ff.to_parquet(os.path.join(cwd,'Test','Hazard_match_sample.parquet'))

In [2]:

import time
time0=time.time() 
Inv=gpd.read_parquet('C:/Users/solei/Documents/Rasters/K3A1.parquet')


In [3]:
import pickle

# Replace 'your_file.pkl' with your actual file path
with open('./ProbDS_cwfd_MaxIter3000.pkl', 'rb') as file:
    data = pickle.load(file)

In [4]:
import numpy as np 
C=np.load('C:/Users/solei/Documents/Rasters/C1A.npy')

In [5]:
nn=40

In [6]:
topn_values = np.partition(C, -nn, axis=1)[:, -nn:][:,::-1]
top3_idx = np.argpartition(C, -nn, axis=1)[:, -nn:]+1

In [7]:
if nn==1:
    topn_values=topn_values*0+1

In [8]:
Inv['ID']=np.arange(0,len(Inv),1)
values = np.array([
    0.000, 0.004, 0.008, 0.012, 0.016, 0.020, 0.024, 0.028, 0.032, 0.036,
    0.040, 0.060, 0.080, 0.100, 0.120, 0.140, 0.160, 0.180, 0.200, 0.220,
    0.240, 0.280, 0.320, 0.360, 0.400, 0.440, 0.480, 0.520, 0.560, 0.600,
    0.640, 0.680, 0.720, 0.760, 0.800, 0.840, 0.880, 0.920, 0.960, 1.000,
    1.040, 1.080, 1.120, 1.160, 1.200, 1.240, 1.280, 1.320, 1.360, 1.400
])

In [9]:
np.ceil(Inv["MWS_OT"].values)

array([2., 2., 2., ..., 2., 2., 2.], shape=(2104234,))

In [12]:
# Extract columns
m_array = Inv["m"].values
id_array = Inv["ID"].values

mws_ot = Inv["MWS_OT"].values
max_in = Inv["Max_In"].values
mws_otu = np.ceil(Inv["MWS_OT"].values).astype(int)
max_inu = np.ceil(Inv["Max_In"].values).astype(int)
mws_otl = np.floor(Inv["MWS_OT"].values).astype(int)
max_inl = np.floor(Inv["Max_In"].values).astype(int)

l=1

Arr=np.zeros((nn,len(Inv),50))
# For each of the 3 top indices
for n in range(top3_idx.shape[1]):
    Inv[str(n)]=0
    col_n = top3_idx[id_array, n]             # shape (num_rows,)
    keys_n = list(zip(m_array, col_n))        # list of (m, col)
    if l==0:
        # Get corresponding data values using vectorized access
        values_n = np.array([
            data[k][i, j] for k, i, j in zip(keys_n, mws_ot.astype(int), max_in.astype(int))
        ])
    else:
        values_u = np.array([
            data[k][i, j] for k, i, j in zip(keys_n, mws_otu, max_inu)
        ]) 

        values_l = np.array([
            data[k][i, j] for k, i, j in zip(keys_n, mws_otl, max_inl)
        ])  

        values_n=values_u * (mws_otu-mws_ot)[:, np.newaxis] +values_l * (mws_ot-mws_otl)[:, np.newaxis] 

    Arr[n,:,:] = values_n


In [11]:
result = np.tensordot(Arr, values, axes=([2], [0])).reshape((len(Inv),nn))  # shape (4, 5)

Inv['DS']= np.sum(result*topn_values,axis=1)

print(np.sum(Inv.DS*Inv['val_struct'])/1000000000)
print(time.time()-time0)

17.25831129617365
86.68724536895752


In [ ]:
topn_values.shape

(2104234, 1)

In [ ]:
# Stack all Inv[str(n)] columns into shape (num_rows, nn, k)
stacked = np.stack([np.vstack(Inv[str(n)].values) for n in range(nn)], axis=1)  # shape: (num_rows, nn, k)
if n==1:
    stacked=stacked.reshape((-1,1)).shape
# Multiply with values (shape: (k,))
dot_products = np.dot(stacked, values)  # shape: (num_rows, nn)

# Weight by topn_values (shape: (num_rows, nn))
Inv['DS'] = np.sum(dot_products * topn_values, axis=1)

ValueError: shapes (2104234,1,1) and (50,) not aligned: 1 (dim 2) != 50 (dim 0)

In [ ]:
nn=1

In [80]:
for n in range(nn):
    Inv[str(n)]=np.array(Inv.apply(lambda row: data[(row['m'], top3_idx[row['ID'],n])][int(row['MWS_OT']),int(row['Max_In'])], axis=1))

ValueError: could not broadcast input array from shape (50,2104234) into shape (1,2104234)

In [23]:
Inv['DS']=np.arange(0,len(Inv),1)*0
if nn==1:
    topn_values=topn_values*0+1
for n in range(nn):
    Inv['DS']+=Inv.apply(lambda row: np.sum(row[str(n)]*values),axis=1)*topn_values[:,n]

ValueError: could not broadcast input array from shape (50,2553709) into shape (1,2553709)

In [20]:
np.sum(Inv.DS*Inv['val_struct'])/1000000000

np.float64(29.23185704663511)

In [162]:
topn_values

array([[0.61965, 0.06885, 0.06885],
       [0.61965, 0.06885, 0.06885],
       [0.61965, 0.06885, 0.06885],
       ...,
       [0.61965, 0.06885, 0.06885],
       [0.61965, 0.06885, 0.06885],
       [0.61965, 0.06885, 0.06885]], shape=(2553709, 3))

In [ ]:
Inv['Dam']*Inv

In [173]:
Inv['0']

0          [0.30124998, 0.15866667, 0.041958336, 0.029708...
1          [0.30124998, 0.15866667, 0.041958336, 0.029708...
2          [0.30124998, 0.15866667, 0.041958336, 0.029708...
3          [0.30124998, 0.15866667, 0.041958336, 0.029708...
4          [0.30124998, 0.15866667, 0.041958336, 0.029708...
                                 ...                        
2553704    [0.39608335, 0.12516667, 0.028875003, 0.020166...
2553705    [0.39608335, 0.12516667, 0.028875003, 0.020166...
2553706    [0.39658335, 0.117375, 0.031333335, 0.02375000...
2553707    [0.39658335, 0.117375, 0.031333335, 0.02375000...
2553708    [0.39841664, 0.117125, 0.030958332, 0.02204166...
Name: 0, Length: 2553709, dtype: object

In [66]:
np.sum(Inv['Dam']*Inv['val_struct'])

np.float64(29234845106.804096)

In [25]:
data[(Y,100)][0,1]

array([0.6035417 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.08108333,
       0.039375  , 0.117125  , 0.040375  , 0.0775    , 0.041     ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ],
      dtype=float32)